KoBERT를 이용해 가장 간단한 모델링

https://velog.io/@8804who/KoBERT%EB%A5%BC-%ED%86%B5%ED%95%9C-%EB%85%B8%EB%9E%98-%EA%B0%80%EC%82%AC-%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D2-KoBERT-%EB%AA%A8%EB%8D%B8-%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0

# 1) 데이터 가져오기

In [1]:
# Matplotlib 와 pandas 그래프에서 한글 사용 설정
import sys

## 노트북이 코랩에서 실행중인지 체크
if 'google.colab' in sys.modules:
  !echo 'debconf debconf/frontend select Noninterative' | debconf-set-selections
  ## 나눔 폰트 설치
  !sudo apt-get -qq -y install fonts-nanum
  import matplotlib.font_manager as fm
  font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
  for fpath in font_files:
    fm.fontManager.addfont(fpath)

  # 나눔바른고딕 폰트 설정
  import matplotlib.pyplot as plt
  plt.rc('font', family='NanumBarunGothic')
  # 마이너스 기호 표시 오류 수정
  import matplotlib
  matplotlib.rcParams['axes.unicode_minus'] = False


debconf: unable to initialize frontend: Noninterative
debconf: (Can't locate Debconf/FrontEnd/Noninterative.pm in @INC (you may need to install the Debconf::FrontEnd::Noninterative module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.34.0 /usr/local/share/perl/5.34.0 /usr/lib/x86_64-linux-gnu/perl5/5.34 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl-base /usr/lib/x86_64-linux-gnu/perl/5.34 /usr/share/perl/5.34 /usr/local/lib/site_perl) at (eval 17) line 2, <> line 1.)
debconf: falling back to frontend: Noninteractive
Selecting previously unselected package fonts-nanum.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import random
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 감성대화 말뭉치
https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86
- 일반인 1,500명 대상의 인터뷰 및 크라우드 소싱 수행
- 우울증 환자 대상 WOZ 대화 수집

=> 전처리 과정으로 "한사람만의 대화를 줄글로 변형 - 질문 제거"

### 1. 데이터 가져오기

In [3]:
train = pd.read_excel("/content/drive/MyDrive/0000/Class/졸프/Dataset/AIhub_감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.xlsx")
valid = pd.read_excel("/content/drive/MyDrive/0000/Class/졸프/Dataset/AIhub_감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation.xlsx")

In [4]:
train_df = train.copy()
valid_df = valid.copy()

### 2. 전처리

In [13]:
# "사람문장3" 결측값 대체
train_df.fillna('',inplace=True)
valid_df.fillna('',inplace=True)

# 문장 병합
train_df['사람문장'] = train_df['사람문장1']+train_df['사람문장2']+train_df['사람문장3']
valid_df['사람문장'] = valid_df['사람문장1']+valid_df['사람문장2']+valid_df['사람문장3']

# "감정 분류"과 "문장"만 추출
train_df = train_df[['사람문장', '감정_대분류']]
valid_df = valid_df[['사람문장', '감정_대분류']]

# 컬럼명 변경
train_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)
valid_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)

# label encoding
le = LabelEncoder()
le=le.fit(train_df['sentiment'])
train_df['target']=le.transform(train_df['sentiment'])

le=le.fit(valid_df['sentiment'])
valid_df['target']=le.transform(valid_df['sentiment'])

<ipython-input-13-314d254c9c6a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.rename(columns={'사람문장': 'text',
<ipython-input-13-314d254c9c6a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df.rename(columns={'사람문장': 'text',
<ipython-input-13-314d254c9c6a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['target']=le.transform(train_df['sentiment'])

In [15]:
train_conv = train_df[['text','target']]
valid_conv = valid_df[['text','target']]

# 2) 데이터 전처리

### 분석기 설치

형태소 분석기 사용으로 Okt 적용. 한국어 형태소 분석을 위해 konlpy와 함께 적용
- Open Korean Text(오픈 한국어 텍스트)의 약자로, 한국어 형태소 분석을 위한 Konlpy의 하위 패키지 중 하나

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 41.3 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt

okt = Okt()
print(okt.morphs('한국어 형태소 분석을 시작합니다.'))

['한국어', '형태소', '분석', '을', '시작', '합니다', '.']


### 제거

In [ ]:
def stopwords(text):
    stop_words = "만 지만 만큼 성큼 그만큼 이만큼 에 등 듯 외 의 정말 것 같다 관한 그래서 그렇게 그렇다 그렇지 때 때문에 때문 를 마저 부터 어느 어떻게 에서 으로서 와 과 할때 할 때 혹시 혹시나 해요 했어요 이다 이 가 을 를 은 는 에게 되다 하다 하다가 뿐 대하여 관하여 과연 그래도 그리고 및 결국 좀 자마자 곧 로부터 동안 해야 그 저 우선"
    stop_words = stop_words.split(' ')

    word_tokens = word_tokenize(text)
    result = []
    for w in word_tokens:
        if w not in stop_words:
            result.append(w)
            n_text = " ".join(result)
    return n_text

In [ ]:
import re
def clean_text(text):
    # 한글 (자음, 모음), 영문(소문자, 대문자), 숫자만 남김, 중복생성된 공백 삭제
    text_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\s.%]", " ", text)
    text_clean = re.sub(" +", " ", text_clean)
    # 문자열 시작과 끝에 있는 공백 제거
    text_clean = text_clean.strip()
    return text_clean

In [ ]:
# 최종 전처리
def word_processing(text):
    # text = chinese_etc(text)
    # text = same_meaning(text)
    text = stopwords(text)
    # text = word_delete(text)
    text = clean_text(text)
    return text

# 3) modeling setting
[Bert 감정분류 모델](https://dacon.io/competitions/official/236027/codeshare/6989?page=1&dtype=recent)

In [17]:
train = train_conv.copy()
valid = valid_conv.copy()

In [18]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from transformers import BertModel
from torch.optim import Adam

import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [19]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [20]:
CFG = {
    'EPOCHS': 3,
    'LEARNING_RATE':1e-5,
    'BATCH_SIZE':8,
    'SEED':41
}

In [21]:
# fixed randomseed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

**토큰화**
- BERT 모델을 사용하기 위해 필요한 토크나이저 정의
- 자연어처리를 위한 사전 훈련된 모델 중 하나
- 텍스트를 토큰으로 분할해 모델에 입력으로 제공하기 위해 토크나이저 필요

한국어 자연어 처리 작업에는 한국어에 특화된 토크나이저와 사전 훈련된 언어 모델을 사용하는 것이 더 적합

=> BERT와 같은 모델은 다양한 언어에 대한 사전훈련이 가능하나 대부분의 경우 해당 언어에 특화된 모델을 사용하는 것이 성능 향상에 도움

KoBERT는 문장을 토큰화하는 과정을 자체적으로 수행하므로, 별도의 토크나이저를 사용할 필요가 없습니다. 대신에 입력 데이터를 KoBERT가 요구하는 형식에 맞게 변환하여 모델에 입력으로 제공하면 됩니다.

In [22]:
# tokenizer define
tokenizers = BertTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [23]:
# custom dataset : 사용자 정의 데이터셋 만듦
class CustomDataset(Dataset):

    def __init__(self, data, mode = "train"):
        self.dataset = data
        self.tokenizer = tokenizers
        self.mode = mode
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset['text'][idx]
        inputs = self.tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
        input_ids = inputs['input_ids'][0]
        token_type_ids = inputs['token_type_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        if self.mode == "train":
            y = self.dataset['target'][idx]
            return input_ids, token_type_ids, attention_mask, y
        else:
            return input_ids, token_type_ids, attention_mask

In [24]:
train = CustomDataset(train, mode = "train")
valid = CustomDataset(valid, mode = "train")

train_dataloader = torch.utils.data.DataLoader(train, batch_size= CFG['BATCH_SIZE'], shuffle=True)
val_dataloader = torch.utils.data.DataLoader(valid, batch_size= CFG['BATCH_SIZE'], shuffle=False)

## model define

In [25]:
class BaseModel(nn.Module):

    def __init__(self, dropout=0.5, num_classes=len(le.classes_)):

        super(BaseModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [26]:
def train(model, optimizer, train_loader, test_loader, device):

    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)

    best_score = 0
    best_model = "None"
    for epoch_num in range(CFG["EPOCHS"]):

        model.train()
        train_loss = []
        for input_ids, token_type_ids, attention_mask, train_label in tqdm(train_loader):
            optimizer.zero_grad()

            train_label = train_label.to(device)
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label.long())
            train_loss.append(batch_loss.item())

            batch_loss.backward()
            optimizer.step()

        val_loss, val_score = validation(model, criterion, test_loader, device)
        print(f'Epoch [{epoch_num}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')

        if best_score < val_score:
            best_model = model
            best_score = val_score

    return best_model

In [27]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()

    val_loss = []
    model_preds = []
    true_labels = []
    with torch.no_grad():
        for input_ids, token_type_ids, attention_mask, valid_label in tqdm(test_loader):
            valid_label = valid_label.to(device)
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, valid_label.long())
            val_loss.append(batch_loss.item())

            model_preds += output.argmax(1).detach().cpu().numpy().tolist()
            true_labels += valid_label.detach().cpu().numpy().tolist()
        val_f1 = competition_metric(true_labels, model_preds)
    return val_loss, val_f1

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])

infer_model = train(model, optimizer, train_dataloader, val_dataloader, device)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

  0%|          | 0/6454 [00:00<?, ?it/s]